In [1]:
import numpy as np
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

C:\Users\aysha\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
file = open('ai_forproject.txt', 'r')
text = file.read()
text

"Artificial Intelligence (AI) and Machine Learning (ML) are two cutting-edge fields in computer science that have the potential to revolutionize the way we live, work, and interact with technology. AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, making decisions, and learning from experience. ML, on the other hand, is a subset of AI that focuses on the development of algorithms and models that allow computers to learn and improve from data without being explicitly programmed.\n\nThese two fields have seen remarkable advancements in recent years, thanks to the availability of vast amounts of data and the increasing processing power of computers. As a result, AI and ML are being integrated into various applications and industries, from healthcare and finance to transportation and entertainment. In this text, we'll explore the fundamental concepts and key develo

In [3]:
def cleaned_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    text = re.sub(r'[^a-z A-Z\.,()0-9]', '', text)
    return text

data = cleaned_text(text).split(".")
data

['Artificial Intelligence (AI) and Machine Learning (ML) are two cuttingedge fields in computer science that have the potential to revolutionize the way we live, work, and interact with technology',
 ' AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, making decisions, and learning from experience',
 ' ML, on the other hand, is a subset of AI that focuses on the development of algorithms and models that allow computers to learn and improve from data without being explicitly programmed',
 '  These two fields have seen remarkable advancements in recent years, thanks to the availability of vast amounts of data and the increasing processing power of computers',
 ' As a result, AI and ML are being integrated into various applications and industries, from healthcare and finance to transportation and entertainment',
 ' In this text, well explore the fundamental concep

In [4]:
tokenizer = Tokenizer(oov_token = '#OOV')
tokenizer.fit_on_texts(data)

In [5]:
tokenizer.word_index

{'#OOV': 1,
 'and': 2,
 'in': 3,
 'ai': 4,
 'the': 5,
 'to': 6,
 'of': 7,
 'data': 8,
 'learning': 9,
 'is': 10,
 'machine': 11,
 'for': 12,
 'from': 13,
 'a': 14,
 'are': 15,
 'can': 16,
 'on': 17,
 'with': 18,
 'uses': 19,
 'models': 20,
 'systems': 21,
 'language': 22,
 'more': 23,
 'by': 24,
 'these': 25,
 'like': 26,
 'management': 27,
 'autonomous': 28,
 'analysis': 29,
 'improving': 30,
 'health': 31,
 'that': 32,
 'as': 33,
 'their': 34,
 'content': 35,
 'human': 36,
 'algorithms': 37,
 'analyze': 38,
 'research': 39,
 'development': 40,
 'applications': 41,
 'identify': 42,
 'design': 43,
 'wildlife': 44,
 'optimizes': 45,
 'energy': 46,
 'natural': 47,
 'making': 48,
 'helps': 49,
 'monitor': 50,
 'analyzes': 51,
 'potential': 52,
 'used': 53,
 'analyzing': 54,
 'safety': 55,
 'ml': 56,
 'improve': 57,
 'use': 58,
 'user': 59,
 'personalized': 60,
 'voice': 61,
 'efficiency': 62,
 'have': 63,
 'patterns': 64,
 'text': 65,
 'or': 66,
 'it': 67,
 'complex': 68,
 'insights': 69,

In [6]:
num_sentences = len(data)
num_words = len(tokenizer.word_index)
print("Total no.of sentences : ",num_sentences)
print("Total no.of words     : ",num_words)

Total no.of sentences :  285
Total no.of words     :  1413


In [7]:
tokenizer.texts_to_sequences([data[0]])

[[274,
  139,
  4,
  2,
  11,
  9,
  56,
  15,
  275,
  645,
  206,
  3,
  207,
  646,
  32,
  63,
  5,
  52,
  6,
  276,
  5,
  208,
  387,
  647,
  648,
  2,
  277,
  18,
  86]]

In [8]:
inputs = []
for sequence in data:
    sequences = tokenizer.texts_to_sequences([sequence])[0]
    
    for i in range(1,len(sequences)):
        inputs.append(sequences[:i+1])

In [9]:
inputs

[[274, 139],
 [274, 139, 4],
 [274, 139, 4, 2],
 [274, 139, 4, 2, 11],
 [274, 139, 4, 2, 11, 9],
 [274, 139, 4, 2, 11, 9, 56],
 [274, 139, 4, 2, 11, 9, 56, 15],
 [274, 139, 4, 2, 11, 9, 56, 15, 275],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207, 646],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207, 646, 32],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207, 646, 32, 63],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207, 646, 32, 63, 5],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207, 646, 32, 63, 5, 52],
 [274, 139, 4, 2, 11, 9, 56, 15, 275, 645, 206, 3, 207, 646, 32, 63, 5, 52, 6],
 [274,
  139,
  4,
  2,
  11,
  9,
  56,
  15,
  275,
  645,
  206,
  3,
  207,
  646,
  32,
  63,
  5,
  52,
  6,
  276],
 [274,
  139,
  4,
  2,
  11,
  9,
  56,
  15,


In [10]:
max_len = max([len(x) for x in inputs])
max_len

32

In [11]:
padded_input_sequences = pad_sequences(inputs,maxlen=max_len,padding = 'pre')
padded_input_sequences

array([[  0,   0,   0, ...,   0, 274, 139],
       [  0,   0,   0, ..., 274, 139,   4],
       [  0,   0,   0, ..., 139,   4,   2],
       ...,
       [  0,   0,   0, ...,   2, 129,  85],
       [  0,   0,   0, ..., 129,  85, 340],
       [  0,   0,   0, ...,  85, 340,  64]])

In [12]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [13]:
X.shape

(4933, 31)

In [14]:
y = to_categorical(y, num_classes = num_words+1)

In [15]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
y.shape

(4933, 1414)

In [70]:
model = Sequential()
model.add(Embedding(num_words+1,100,input_length=max_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(num_words+1,activation = 'softmax'))

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 31, 100)           141400    
                                                                 
 bidirectional (Bidirection  (None, 300)               301200    
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 1414)              425614    
                                                                 
Total params: 868214 (3.31 MB)
Trainable params: 868214 (3.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [73]:
model.compile(loss="categorical_crossentropy",
             optimizer = 'adam',
             metrics = ['accuracy'])

In [74]:
model.fit(X,y,
          epochs = 100,
          verbose = 1,
          validation_split=0.2
          #callbacks = [early_stopping]
         )

Epoch 1/100
124/124 [==============================] - 31s 206ms/step - loss: 6.5700 - accuracy: 0.0839 - val_loss: 6.4865 - val_accuracy: 0.1581
Epoch 2/100
124/124 [==============================] - 26s 207ms/step - loss: 5.9970 - accuracy: 0.1072 - val_loss: 6.4478 - val_accuracy: 0.1581
Epoch 3/100
124/124 [==============================] - 26s 214ms/step - loss: 5.7566 - accuracy: 0.1087 - val_loss: 6.5058 - val_accuracy: 0.1621
Epoch 4/100
124/124 [==============================] - 26s 213ms/step - loss: 5.4738 - accuracy: 0.1290 - val_loss: 6.5530 - val_accuracy: 0.1621
Epoch 5/100
124/124 [==============================] - 26s 213ms/step - loss: 5.1501 - accuracy: 0.1483 - val_loss: 6.6126 - val_accuracy: 0.1733
Epoch 6/100
124/124 [==============================] - 27s 214ms/step - loss: 4.8150 - accuracy: 0.1703 - val_loss: 6.7286 - val_accuracy: 0.1682
Epoch 7/100
124/124 [==============================] - 26s 213ms/step - loss: 4.5035 - accuracy: 0.1850 - val_loss: 6.7640 -

Epoch 57/100
124/124 [==============================] - 16s 132ms/step - loss: 0.2501 - accuracy: 0.9394 - val_loss: 9.5732 - val_accuracy: 0.1449
Epoch 58/100
124/124 [==============================] - 18s 142ms/step - loss: 0.2483 - accuracy: 0.9372 - val_loss: 9.5832 - val_accuracy: 0.1469
Epoch 59/100
124/124 [==============================] - 19s 155ms/step - loss: 0.2463 - accuracy: 0.9374 - val_loss: 9.6296 - val_accuracy: 0.1489
Epoch 60/100
124/124 [==============================] - 19s 151ms/step - loss: 0.2418 - accuracy: 0.9387 - val_loss: 9.6577 - val_accuracy: 0.1499
Epoch 61/100
124/124 [==============================] - 20s 162ms/step - loss: 0.2395 - accuracy: 0.9382 - val_loss: 9.7034 - val_accuracy: 0.1439
Epoch 62/100
124/124 [==============================] - 22s 180ms/step - loss: 0.2392 - accuracy: 0.9369 - val_loss: 9.7324 - val_accuracy: 0.1489
Epoch 63/100
124/124 [==============================] - 25s 201ms/step - loss: 0.2371 - accuracy: 0.9382 - val_loss: 9

In [64]:
text = "who is a data scientist"

token = tokenizer.texts_to_sequences([text])[0]
pad_seq = pad_sequences([token],maxlen=max_len-1,padding='pre')
pad_seq

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 10, 14,  8,  1]])

In [65]:
pred = model.predict(pad_seq)
pred

1/1 [==============================] - 0s 52ms/step


array([[2.2512399e-08, 2.8684711e-08, 1.7587668e-08, ..., 3.5138136e-08,
        2.9960116e-08, 2.7703136e-08]], dtype=float32)

In [66]:
pred.shape

(1, 1414)

In [67]:
pos = np.argmax(pred)
pos

15

In [68]:
for i in range(20):
    token = tokenizer.texts_to_sequences([text])[0] 
    pad_seq = pad_sequences([token],maxlen=max_len-1,padding='pre')
    pos = np.argmax(model.predict(pad_seq))
    
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)

1/1 [==============================] - 0s 50ms/step
who is a data scientist are
1/1 [==============================] - 0s 40ms/step
who is a data scientist are trained
1/1 [==============================] - 0s 40ms/step
who is a data scientist are trained on
1/1 [==============================] - 0s 40ms/step
who is a data scientist are trained on ai
1/1 [==============================] - 0s 42ms/step
who is a data scientist are trained on ai that
1/1 [==============================] - 0s 37ms/step
who is a data scientist are trained on ai that focuses
1/1 [==============================] - 0s 35ms/step
who is a data scientist are trained on ai that focuses on
1/1 [==============================] - 0s 39ms/step
who is a data scientist are trained on ai that focuses on the
1/1 [==============================] - 0s 32ms/step
who is a data scientist are trained on ai that focuses on the interaction
1/1 [==============================] - 0s 36ms/step
who is a data scientist are trained on 